# Financial Analyst Agent Using CrewAI and LlamaIndex

### Install required dependencies

In [19]:
!pip install llama-index --quiet
!pip install llama-index-llms-groq --quiet
!pip install llama-index-core --quiet
!pip install llama-index-readers-file --quiet
!pip install llama-index-tools-wolfram-alpha --quiet
!pip install llama-index-embeddings-huggingface --quiet
!pip install 'crewai[tools]' --quiet

### Let's setup the LLM with Groq. Sign up at Groq, it is free. Go to Groq console and get the API keys.

In [20]:
from llama_index.llms.groq import Groq

In [25]:
llm = Groq(model="llama3-70b-8192", api_key="Add Your Groq API Key")

In [26]:
response = llm.complete("Explain the importance of LLMs")
print(response)

Large Language Models (LLMs) have revolutionized the field of Natural Language Processing (NLP) and have numerous applications across various industries. The importance of LLMs can be summarized as follows:

1. **Improved Language Understanding**: LLMs have demonstrated remarkable capabilities in understanding human language, including nuances, context, and subtleties. This enables them to generate more accurate and informative responses, making them useful for applications like chatbots, virtual assistants, and language translation.
2. **Enhanced Text Generation**: LLMs can generate coherent, context-specific, and often indistinguishable-from-human text. This has opened up new possibilities for content creation, such as automated writing, summarization, and even creative writing.
3. **Advancements in AI Research**: LLMs have pushed the boundaries of AI research, enabling the development of more sophisticated models that can learn from large datasets and improve over time. This has led

### Crew AI requires a chat based model for binding

In [27]:
from langchain_openai import ChatOpenAI

# Define your API key here
groq_api_key = "Add Your Groq API Key"

chat_llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=groq_api_key,
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=1000,
)

### Download data

In [28]:
!wget "https://www.infosys.com/investors/reports-filings/annual-report/annual/documents/infosys-ar-23.pdf" -O infy_nav.pdf

--2024-07-13 15:12:04--  https://www.infosys.com/investors/reports-filings/annual-report/annual/documents/infosys-ar-23.pdf
Resolving www.infosys.com (www.infosys.com)... 23.9.129.136
Connecting to www.infosys.com (www.infosys.com)|23.9.129.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘infy_nav.pdf’

infy_nav.pdf            [  <=>               ]  12.52M  49.7MB/s    in 0.3s    

2024-07-13 15:12:20 (49.7 MB/s) - ‘infy_nav.pdf’ saved [13133646]



### Parse the content of the data

In [35]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI
import os
from langchain_openai import ChatOpenAI

In [36]:
reader = SimpleDirectoryReader(input_files=["infy_nav.pdf"])
docs = reader.load_data()
docs[1]

Document(id_='1fac8a5d-dce6-4f9b-a1ed-86e9bfcc52d4', embedding=None, metadata={'page_label': '2', 'file_name': 'infy_nav.pdf', 'file_path': 'infy_nav.pdf', 'file_type': 'application/pdf', 'file_size': 13133646, 'creation_date': '2024-07-13', 'last_modified_date': '2023-06-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='The context surrounding \nan enterprise, created, \nand influenced by multiple \ninherently uncertain forces, can \nsignificantly impact the fortunes \nof a business. While this volatility \nhas come to be expected as normal, \nnot every business emerges from it having \ntackled the situation with the same resilience. \nThose that get a head start in preparing and taking \non the challenges are also the ones that come out of the \nun

### Setup the embedding model

In [37]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [38]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

### Build Index

In [39]:
index = VectorStoreIndex.from_documents(docs,
                                        embed_model=embed_model,
                                        )

### Build the Query Engine

In [44]:
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

### Instantiate the query engine as a Tool

In [49]:
from crewai_tools import LlamaIndexTool
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="infy Query Tool",
    description="Use this tool to lookup the 2022-23 Infosys Annual Report",
)

In [50]:
query_tool.args_schema.schema()

{'title': 'QueryToolSchema',
 'description': 'Schema for query tool.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'Search query for the query tool.',
   'type': 'string'}},
 'required': ['query']}

### Instantiate Researcher and Writer Agents

In [51]:
import os
from crewai import Agent, Task, Crew, Process

In [56]:
researcher = Agent(
    role="Lead Financial Analyst",
    goal="Uncover insights about different tech companies",
    backstory="""You work at an asset management firm.
  Your goal is to understand tech stocks like Infosys.""",
    verbose=True,
    allow_delegation=False,
    tools=[query_tool],
    llm=chat_llm,
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    llm=chat_llm,
    verbose=True,
    allow_delegation=False,
)

### Define respective tasks

In [61]:
task1 = Task(
    description="""Conduct a comprehensive analysis of Infosys's use of artificial intelligence in their integrated annual report 2022-23.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the importance of artificial intelligence at Infosys.
  Your post should be informative yet accessible, catering to a casual audience.
  Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

### Instantiate the Crew with a sequential process

In [62]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=2,  # You can set it to 1 or 2 to different logging levels
)

### Initiate the agent workflow: Let the magic begin!

In [63]:
result = crew.kickoff()

print("######################")
print(result)

 [2024-07-13 15:25:42][DEBUG]: == Working Agent: Lead Financial Analyst
 [2024-07-13 15:25:42][INFO]: == Starting Task: Conduct a comprehensive analysis of Infosys's use of artificial intelligence in their integrated annual report 2022-23.


> Entering new CrewAgentExecutor chain...
Thought: I need to understand Infosys's use of artificial intelligence in their integrated annual report 2022-23, so I should start by looking up the 2022-23 Infosys Annual Report.

Action: infy Query Tool
Action Input: {"query": "2022-23 Infosys Annual Report"} 

The provided context information appears to be the Infosys Integrated Annual Report for 2022-23.

Thought: Now that I have the 2022-23 Infosys Annual Report, I should search for keywords related to artificial intelligence to understand Infosys's use of AI.

Action: infy Query Tool
Action Input: {"query": "artificial intelligence"} 

In today's uncertain landscape, companies must become more resilient and adaptable to thrive. Infosys is embracing t